Задание:
Используя данные из train.csv, построить
модель для предсказания цен на недвижимость (квартиры).
С помощью полученной модели предсказать
цены для квартир из файла test.csv.

Целевая переменная:
Price

Метрика:
R2 - коэффициент детерминации (sklearn.metrics.r2_score)

Сдача проекта:
1. Прислать в раздел Задания Урока 10 ("Вебинар. Консультация по итоговому проекту")
ссылку на программу в github (программа должна содержаться в файле Jupyter Notebook 
с расширением ipynb). (Pull request не нужен, только ссылка ведущая на сам скрипт).
2. Приложить файл с названием по образцу SShirkin_predictions.csv
с предсказанными ценами для квартир из test.csv (файл должен содержать два поля: Id, Price).
В файле с предсказаниями должна быть 5001 строка (шапка + 5000 предсказаний).

Сроки и условия сдачи:
Дедлайн: сдать проект нужно в течение 72 часов после начала Урока 10 ("Вебинар. Консультация по итоговому проекту").
Для успешной сдачи должны быть все предсказания (для 5000 квартир) и R2 должен быть больше 0.6.
При сдаче до дедлайна результат проекта может попасть в топ лучших результатов.
Повторная сдача и проверка результатов возможны только при условии предыдущей неуспешной сдачи.
Успешный проект нельзя пересдать в целях повышения результата.

In [39]:
import warnings

warnings.filterwarnings('ignore')

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
from scipy.stats import norm, normaltest
#from sklearn.model_selection import train_test_split
from sklearn import set_config


In [41]:
df_base = pd.read_csv('/home/mikhail/Рабочий стол/Geek_Script_2605/Kurs_project_task/train.csv')
#df_tar = pd.read_csv('/home/mikhail/Рабочий стол/Geek_Script_2605/Kurs_project_task/train.csv')
# df = pd.read_csv('/home/mikhail/Рабочий стол/Geek_Script_2605/Geek_Home_DC/creditcard.csv')

In [42]:
df_base.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


Причесываем df_base


In [43]:
df_base.loc[df_base['LifeSquare'].isnull(), 'LifeSquare2'] = df_base['Square']

In [44]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
LifeSquare2      2113 non-null float64
dtypes: float64(9), int64(9), object(3)
memory usage: 1.6+ MB


In [45]:
df_base = df_base.loc[:, list(df_base.columns[:5]) + ['LifeSquare2'] + list(df_base.columns[5:-1])]

In [46]:
df_base.loc[df_base['LifeSquare'].isnull(), 'LifeSquare'] = df_base['Square'] - 3

In [47]:
df_base['Healthcare_1'] = df_base['Healthcare_1'].fillna(-999)

In [48]:
df_base = df_base.drop('LifeSquare2', axis=1)

In [49]:
df_base.drop('Healthcare_1', axis=1, inplace=True)

In [50]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(7), int64(9), object(3)
memory usage: 1.4+ MB


In [51]:
df_base = pd.get_dummies(df_base)

In [52]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Price            10000 non-null float64
Ecology_2_A      10000 non-null uint8
Ecology_2_B      10000 non-null uint8
Ecology_3_A      10000 non-null uint8
Ecology_3_B      10000 non-null uint8
Shops_2_A        10000 non-null uint8
Shops_2_B        10000 non-null uint8
dtypes: float64(7), int64(9), uint8(6)
mem

In [71]:
df_base.corr()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
Id,1.000000,0.012973,-0.005847,-0.010071,0.012864,0.019880,0.001348,-0.008376,0.005004,0.018097,...,-0.009358,0.001502,-0.008718,0.009880,-0.003099,0.003099,0.005408,-0.005408,-0.003645,0.003645
DistrictId,0.012973,1.000000,0.071432,-0.026613,-0.043978,0.040358,-0.120373,-0.149051,0.013430,0.065294,...,0.136095,0.306147,0.174214,0.265100,0.093406,-0.093406,0.027902,-0.027902,0.021018,-0.021018
Rooms,-0.005847,0.071432,1.000000,0.662893,0.149181,0.005123,-0.000665,-0.029302,-0.010612,-0.032347,...,0.012811,0.063557,0.053618,0.550291,0.001971,-0.001971,0.005909,-0.005909,0.000146,-0.000146
Square,-0.010071,-0.026613,0.662893,1.000000,0.244232,0.008320,0.114791,0.081505,-0.009032,-0.064479,...,0.035241,-0.022960,0.021357,0.520075,-0.029926,0.029926,-0.022748,0.022748,0.049380,-0.049380
LifeSquare,0.012864,-0.043978,0.149181,0.244232,1.000000,-0.010519,0.035819,0.020830,-0.002726,-0.030678,...,0.013186,-0.046321,-0.018892,0.080774,-0.011100,0.011100,-0.017649,0.017649,0.018417,-0.018417
KitchenSquare,0.019880,0.040358,0.005123,0.008320,-0.010519,1.000000,-0.011397,0.000780,0.000958,-0.005622,...,-0.015140,0.043170,0.010216,0.028864,-0.000268,0.000268,0.007919,-0.007919,0.017315,-0.017315
Floor,0.001348,-0.120373,-0.000665,0.114791,0.035819,-0.011397,1.000000,0.418986,0.000928,-0.016133,...,-0.002237,-0.065537,0.024264,0.128715,-0.049661,0.049661,-0.025767,0.025767,0.013626,-0.013626
HouseFloor,-0.008376,-0.149051,-0.029302,0.081505,0.020830,0.000780,0.418986,1.000000,-0.000864,-0.004362,...,-0.008137,-0.068728,0.026279,0.088280,-0.059347,0.059347,-0.009258,0.009258,-0.051002,0.051002
HouseYear,0.005004,0.013430,-0.010612,-0.009032,-0.002726,0.000958,0.000928,-0.000864,1.000000,0.001465,...,0.000819,0.011245,0.003681,0.004305,-0.000997,0.000997,-0.001686,0.001686,-0.002994,0.002994
Ecology_1,0.018097,0.065294,-0.032347,-0.064479,-0.030678,-0.005622,-0.016133,-0.004362,0.001465,1.000000,...,-0.124068,0.030873,-0.076749,-0.058381,0.110725,-0.110725,0.223197,-0.223197,-0.079809,0.079809


In [53]:
df_base['Price'].describe()

count     10000.000000
mean     214138.857399
std       92872.293865
min       59174.778028
25%      153872.633942
50%      192269.644879
75%      249135.462171
max      633233.466570
Name: Price, dtype: float64

In [54]:
target = df_base['Price']


                       Ощущение, что "Х" у меня с неполными параметрами:

In [55]:
X = pd.DataFrame(df_base)

X.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Helthcare_2,Shops_1,Price,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,...,5,0,11,184966.930730,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,...,1,1,16,300009.450063,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,...,0,1,3,220925.908524,0,1,0,1,0,1
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,...,3,0,5,175616.227217,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,...,1,2,4,150226.531644,0,1,0,1,0,1


In [69]:
X = pd.DataFrame(df_base)

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Price            10000 non-null float64
Ecology_2_A      10000 non-null uint8
Ecology_2_B      10000 non-null uint8
Ecology_3_A      10000 non-null uint8
Ecology_3_B      10000 non-null uint8
Shops_2_A        10000 non-null uint8
Shops_2_B        10000 non-null uint8
dtypes: float64(7), int64(9), uint8(6)
mem

In [57]:
y = pd.DataFrame(target, columns=['Price'])

y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
Price    10000 non-null float64
dtypes: float64(1)
memory usage: 78.2 KB


                                         Делаем тренировочную модель

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [60]:
from sklearn.linear_model import LinearRegression

In [61]:
lr = LinearRegression()


In [62]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [63]:
y_pred = lr.predict(X_test)

y_pred.shape

(2500, 1)

In [66]:
check_test = pd.DataFrame({
    "y_test": y_test['Price'],
    "y_pred": y_pred.flatten(),
})

check_test.head(50)

,y_test,y_pred
3695,179294.237176,179294.237176
9368,343643.748717,343643.748717
8675,204259.631582,204259.631582
8819,77566.751621,77566.751621
2863,109315.261198,109315.261198
3150,202906.219573,202906.219573
5652,259654.497067,259654.497067
700,120724.056008,120724.056008
8617,217361.700539,217361.700539
9028,107458.639657,107458.639657


In [70]:
check_test["error"] = check_test["y_pred"] - check_test["y_test"]

check_test.head()

,y_test,y_pred,error
3695,179294.237176,179294.237176,-8.731149e-11
9368,343643.748717,343643.748717,5.820766e-11
8675,204259.631582,204259.631582,0.000000e+00
8819,77566.751621,77566.751621,-1.600711e-10
2863,109315.261198,109315.261198,-1.455192e-10


                                            Смотрим среднюю квадратичную ошибку

In [79]:
mse1 = (check_test["error"] ** 2).mean()

mse1

1.2846292260471852e-20

                                              Смотрим абсолютную ошибку

In [80]:
(np.abs(check_test["error"])).mean()

9.053328540176153e-11

In [81]:
lr.intercept_

array([-2.61934474e-10])